In [1]:
# model

import tensorflow as tf

# (8, 3)
x_data = [[1, 2, 1],
          [1, 3, 2],
          [1, 3, 4],
          [1, 5, 5],
          [1, 7, 5],
          [1, 2, 5],
          [1, 6, 6],
          [1, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

x_test = [[2, 1, 1],
          [3, 1, 2],
          [3, 3, 4]]
y_test = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1]]

X = tf.placeholder(tf.float32, [None, 3])
Y = tf.placeholder(tf.float32, [None, 3])
W = tf.Variable(tf.random_normal([3, 3]))
b = tf.Variable(tf.random_normal([3]))

hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

prediction = tf.argmax(hypothesis, 1)
accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, tf.argmax(Y, 1)), tf.float32))

In [13]:
# learning

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(201):
        cost_val, W_val, _ = sess.run([cost, W, optimizer], feed_dict={X:x_data, Y:y_data})
        print(step, cost_val)
        
    # accuracy
    print("Prediction:", sess.run(prediction, feed_dict={X:x_test}))
    print("Accuracy:", sess.run(accuracy, feed_dict={X:x_test, Y:y_test}))

0 7.5454655
1 4.883936
2 4.0577803
3 3.2568867
4 2.498672
5 1.8597841
6 1.5730894
7 1.5390842
8 1.5180362
9 1.4998242
10 1.4822818
11 1.4651506
12 1.4483788
13 1.4319541
14 1.4158704
15 1.4001232
16 1.3847077
17 1.3696189
18 1.3548524
19 1.3404028
20 1.3262652
21 1.3124343
22 1.298905
23 1.285672
24 1.2727296
25 1.2600727
26 1.2476954
27 1.2355931
28 1.2237594
29 1.212189
30 1.2008766
31 1.1898167
32 1.1790037
33 1.1684324
34 1.1580973
35 1.1479931
36 1.1381141
37 1.1284556
38 1.1190124
39 1.1097789
40 1.1007504
41 1.0919224
42 1.0832894
43 1.0748466
44 1.0665896
45 1.0585136
46 1.0506141
47 1.0428866
48 1.0353268
49 1.0279305
50 1.0206935
51 1.0136116
52 1.0066807
53 0.99989736
54 0.9932572
55 0.986757
56 0.9803929
57 0.97416127
58 0.9680587
59 0.962082
60 0.95622766
61 0.9504926
62 0.9448735
63 0.9393678
64 0.9339721
65 0.92868364
66 0.9234997
67 0.9184176
68 0.9134345
69 0.90854794
70 0.9037554
71 0.8990544
72 0.89444274
73 0.88991797
74 0.8854778
75 0.88112015
76 0.876843
77 0.8726

Learning rate 이 잘못 설정되면...
* Too large : Overshooting -> cost 가 inf, nan 이 됨
* Too small : 학습이 안되거나 Local Minimum 으로 학습

Non-normalized inputs
* x1, x2 간의 값이 차이가 클 경우
* 긴 타원형 Graph
* cost : nan
Solution
* Nomalize
* MinMaxScaler : Min 을 0. 으로 Max 를 1. 으로 

In [18]:
import tensorflow as tf
import numpy as np

def MinMaxScaler(data):
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    return numerator / (denominator + 1e-7)

xy = np.array([[828.659973, 833.450012, 908100, 828.349976, 831.659973],
               [823.02002, 828.070007, 1828100, 821.655029, 828.070007],
               [819.929993, 824.400024, 1438100, 818.97998, 824.159973],
               [816, 820.958984, 1008100, 815.48999, 819.23999],
               [819.359985, 823, 1188100, 818.469971, 818.97998],
               [819, 823, 1198100, 816, 820.450012],
               [811.700012, 815.25, 1098100, 809.780029, 813.669983],
               [809.51001, 816.659973, 1398100, 804.539978, 809.559998]])

# MinMaxScaler 로 Overshooting 방지
xy = MinMaxScaler(xy)
print(xy)

x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

X = tf.placeholder(tf.float32, [None, 4])
Y = tf.placeholder(tf.float32, [None, 1])
W = tf.Variable(tf.random_normal([4, 1]))
b = tf.Variable(tf.random_normal([1]))

hypothesis = tf.matmul(X, W) + b
cost = tf.reduce_mean(tf.square(hypothesis - Y))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5).minimize(cost)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(101):
        cost_val, hy_val, _ = sess.run([cost, hypothesis, optimizer], feed_dict={X: x_data, Y: y_data})
        print(step, "Cost:", cost_val)


[[0.99999999 0.99999999 0.         1.         1.        ]
 [0.70548491 0.70439552 1.         0.71881782 0.83755791]
 [0.54412549 0.50274824 0.57608696 0.606468   0.6606331 ]
 [0.33890353 0.31368023 0.10869565 0.45989134 0.43800918]
 [0.51436    0.42582389 0.30434783 0.58504805 0.42624401]
 [0.49556179 0.42582389 0.31521739 0.48131134 0.49276137]
 [0.11436064 0.         0.20652174 0.22007776 0.18597238]
 [0.         0.07747099 0.5326087  0.         0.        ]]
0 Cost: 1.0950954
1 Cost: 1.0950146
2 Cost: 1.0949339
3 Cost: 1.0948533
4 Cost: 1.0947726
5 Cost: 1.0946919
6 Cost: 1.0946112
7 Cost: 1.0945305
8 Cost: 1.0944498
9 Cost: 1.0943692
10 Cost: 1.0942885
11 Cost: 1.0942078
12 Cost: 1.0941272
13 Cost: 1.0940466
14 Cost: 1.093966
15 Cost: 1.0938854
16 Cost: 1.0938048
17 Cost: 1.0937243
18 Cost: 1.0936438
19 Cost: 1.0935631
20 Cost: 1.0934825
21 Cost: 1.0934021
22 Cost: 1.0933216
23 Cost: 1.093241
24 Cost: 1.0931605
25 Cost: 1.09308
26 Cost: 1.0929995
27 Cost: 1.092919
28 Cost: 1.0928385